In [1]:
import numpy as np
import cv2
import os
import math as mt

In [9]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [3]:
usernames = []

In [4]:
def getkey(item):
    return item[0]

def distance(x1, x2):
    d0, d1, d2 = 0, 0, 0
    for i in range(len(x1)):
        for j in range(len(x1[i])):
            d0 += (int(x1[i][j][0])-int(x2[i][j][0]))**2
            d1 += (int(x1[i][j][1])-int(x2[i][j][1]))**2
            d2 += (int(x1[i][j][2])-int(x2[i][j][2]))**2
    d = d0 + d1 + d2
    return d

def KNN(x, k):
    d = []
    y_train = []
    for file in os.listdir('photos'):
        for photo in os.listdir('photos/' + file):
            img = cv2.imread('photos/' + file + '/' + photo)
            y_train.append(int(file))
            d.append([distance(x, img), int(file)])
    d = sorted(d, key = getkey)
    num_of_classes = len(np.unique(y_train))
    nearest_neighbours = [d[i][1] for i in range(k)]
    output = 0
    max_count = nearest_neighbours.count(0)
    for i in range(1, num_of_classes):
        c = nearest_neighbours.count(i)
        if c > max_count:
            max_count = c
            output = i
    return output

In [5]:
def create_new_user(photos, username):
    # Make a new folder for the user
    os.makedirs('photos/' + str(len(usernames)))
    # Add all the photos in the newly created folder
    for i in range(len(photos)):
        cv2.imwrite('photos/' + str(len(usernames)) + '/' + str(i) + '.jpg', photos[i])
    usernames.append(username)
    

def register():
    # Take the username of the new user
    flag = False
    print("Enter your name")
    while flag == False:
        username = input()
        if username in usernames:
            print("Username already taken, try another one")
        else:
            flag = True
    # click photos for the training set
    photos = []
    cap = cv2.VideoCapture(0)
    num_of_photos = 0
    while num_of_photos < 25:
        ret, frame = cap.read()
        if ret == False:
            continue
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray_frame, 1.3, 5)
        for (x,y,w,h) in faces:
            cv2.rectangle(frame,(x,y),(x+w,y+h),(55,35,100),6)
            roi = frame[y:y+h, x:x+w]
        cv2.imshow('video', frame)
        k = cv2.waitKey(1) & 0xFF
        if k == ord('c'):
            if len(faces) != 1:
                print("Only one face")
                continue
            photos.append(cv2.resize(roi, (150, 150)))
            num_of_photos += 1
            print(num_of_photos)
        elif k == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
    if num_of_photos == 25:
        create_new_user(photos, username)
    else:
        print("Try again, not enough photos were clicked!")

In [6]:
def login():
    photo_clicked = False
    cap = cv2.VideoCapture(0)
    counter = 0
    while True:
        ret, frame = cap.read()
        if ret == False:
            continue
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray_frame, 1.3, 5)
        for (x,y,w,h) in faces:
            roi = frame[y:y+h, x:x+w]
            cv2.rectangle(frame,(x,y),(x+w,y+h),(55,35,100),6)
        cv2.imshow('video', frame)
        k = cv2.waitKey(1) & 0xFF
        if k == ord('c'):
            if len(faces) != 1:
                print("Only one face!")
                continue
            photo_clicked = True
            break
        if k == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
    if photo_clicked == False:
        print("You need to click a photo to authenticate!")
    else:
        ROI = cv2.resize(roi, (150,150))
        k = 15
        match = usernames[KNN(ROI, k)]
        print("Hi", match)

In [16]:
register()

Enter your name
nitin kumar
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25


In [8]:
login()

Hi Nitin Kumar
